**Sommario**
- [Files in Python](#files-in-python)
  - [Aprire e chiudere lo *stream*](#aprire-e-chiudere-lo-*stream*)
  - [Metodi per leggere](#metodi-per-leggere)
  - [Metodi per scrivere](#metodi-per-scrivere)
  - [Metodi per spostarsi nello *stream*](#metodi-per-spostarsi-nello-*stream*)
  - [Percorsi relativi e Current Working Directory](#percorsi-relativi-e-current-working-directory)
    - [Path della Current Working Directory (CWD)](#path-della-current-working-directory-cwd)
    - [Path relativo al path dello script](#path-relativo-al-path-dello-script)
      - [Modulo `os`](#modulo-os)
      - [Modulo `pathlib` (consigliato)](#modulo-pathlib-consigliato)

# Files in Python

## Aprire e chiudere lo *stream*

| Metodo     | Descrizione     |
|------------|-----------------|
| `open(path, mode='rt', encoding=None, newline=None)` | Apre file e ritorna un corrispondente oggetto file-like. Se il file non può essere aperto, viene sollevato un `OSError`. |
| `.close()` | Esegue il flush e chiude lo *stream*. Questo metodo non ha effetto se il file è già chiuso. Una volta che il file è chiuso, qualsiasi operazione sul file (es. leggere o scrivere) solleverà un `ValueError`. |

Possiamo aprire e chiudere i file manualmente:

```python
my_file = open('my_file.txt', mode=<modalità>)
...
my_file.<metodo>()
...
my_file.close
```

Oppure usare un context manager, che pensa lui alla chiusura e alla gestione degli errori:

```python
with open('my_file.txt', mode=<modalità>) as my_file:
    ...
    my_file.<metodo>()
    ...
```

Ciascuna *modalità* (`mode`) ha le proprie peculiarità:

| `mode` | R/W | Posizione iniziale lettura | Punto di inserimento | Se file non esiste | Se file esiste           |
|:------:|:---:|:--------------------------:|:--------------------:|:------------------:|--------------------------|
| `r`    | R   | inizio                     | -                    | errore             | legge                    |
| `r+`   | RW  | inizio                     | inizio e `seek()`    | errore             | modifica                 |
| `w`    | W   | -                          | (inizio e) `seek()`  | crea               | sovrascrive (*truncate*) |
| `w+`   | RW  | (inizio)                   | (inizio e) `seek()`  | crea               | sovrascrive (*truncate*) |
| `a`    | W   | -                          | fine                 | crea               | appende al fondo         |
| `a+`   | RW  | fine                       | fine                 | crea               | appende al fondo         |

## Metodi per leggere

Metodi di un oggetto file-like

| Metodo        | Descrizione     |
|---------------|-----------------|
| `.readable()` | Ritorna `True` se si può leggere dal flusso. Se restituisce `False`, `.read()` solleverà `OSError`. |
| `.read(size)` | Legge e ritorna fino a `size` byte. Se l'argomento è omesso, `None`, o negativo, i dati vengono letti e ritornati fino al raggiungimento della EOF (End-Of-File). Un oggetto `bytes` vuoto viene ritornato se lo *stream* è già alla EOF. |
| `.readline(size)` | Legge e ritorna una linea dallo *stream*. Se `size` è specificato, al massimo verranno letti `size` byte. |
| `.readlines(hint)` | Legge e ritorna una lista di linee dal dallo *stream*. `hint` può essere specificato per controllare il numero di linee lette: non verranno lette più linee se la dimensione totale (in byte) di tutte le linee finora supera `hint`. |

## Metodi per scrivere

| Metodo         | Descrizione     |
|----------------|-----------------|
| `.writable()`  | Ritorna `True` se lo *stream* supporta la scrittura. Se `False`, `.write()` e `.truncate()` solleveranno `OSError`. |
| `.write(string)`  | Scrive l'oggetto `string` (*byte-like*) e ritorna il numero di byte scritti (sempre uguale alla lunghezza di `string` in byte, poiché se la scrittura fallisce verrà sollevato un `OSError`). A seconda dell'implementazione effettiva, questi byte possono essere immediatamente scritti sul flusso sottostante o mantenuti in un buffer per motivi di prestazione e latenza. |
| `.writelines(lines)` | Scrive sullo *stream* un iterabile `lines` contenente linee. I separatori di linea non vengono aggiunti, quindi è usuale che ciascuna delle linee fornite abbia un separatore di linea `\n` alla fine. |
| `.truncate(size=None)` | Ridimensiona lo *stream* alla dimensione `size` data in byte (o alla posizione corrente se `size` non è specificato). |
| `.flush()`     | Svuota (*flush*) i buffer di scrittura dello *stream* se applicabile. Questo non fa nulla per gli *stream* in sola lettura e non bloccanti (asincroni). |

## Metodi per spostarsi nello *stream*

| Metodo     | Descrizione     |
|------------|-----------------|
| `.seekable()` | Return True if the stream supports random access. If False, seek(), tell() and truncate() will raise OSError. |
| `.seek(offset, whence=0)` | Change the stream position to the given byte offset and return the new absolute position. `offset` is interpreted relative to the position indicated by `whence`. The default value for `whence` is 0. Values for whence are: `0` &rarr; start (default); offset 0 +. `1` &rarr; current position; offset + -. `2` &rarr; end; offset -. |
| `.tell()` | Return the current stream position. |

## Percorsi relativi e Current Working Directory

Se eseguiamo uno script in questo modo:

`C:\Users> py C:\Users\Utente\qualunque_cartella\mio_script.py`

Allora, in questo caso:

- la working directory è `C:\Users\`
- la cartella dove si trova lo script è `C:\Users\Utente\qualunque_cartella\`

### Path della Current Working Directory (CWD)

Per ottenere la CWD abbiamo due alternative:

```python
import os

os.getcwd()
```

oppure, più consigliato:


```python
from pathlib import Path

Path.cwd()
```

### Path relativo al path dello script

```python
..
│
└── qualunque_cartella/
    │
    ├── mio_script.py  # Questo è il nostro script.
    │
    └── data/
        │
        └── records.txt  # Questo è il file che voglio raggiungere.
```

#### Modulo `os`

File `mio_script.py`:

```python
import os

# Ottiene il percorso assoluto dello script corrente
script_path = os.path.abspath(__file__)

# Ottiene la directory in cui risiede lo script
script_dir = os.path.dirname(script_path)

# Costruisce un percorso assoluto al file che voglio raggiungere, combinando
# la directory dello script con il percorso relativo al file.
file_absolute_path = os.path.join(script_dir, 'data/records.txt')
```

#### Modulo `pathlib` (consigliato)

File `mio_script.py`:

```python
from pathlib import Path

# Ottiene il percorso assoluto dello script corrente come oggetto Path
script_path = Path(__file__).resolve()

# Ottiene la directory in cui risiede lo script
script_dir = script_path.parent

# Costruisce un percorso assoluto al file che voglio raggiungere, combinando
# la directory dello script con il percorso relativo al file.
file_absolute_path = script_dir / 'data' / 'records.txt'
```